In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

# Initialize the Ollama Model (adjust 'model' name as per your local Ollama setup)
llm = ChatOllama(model="gemma3:1b")  # or any model you've pulled like "mistral", "llama2", etc.

# Define System Prompt
system_prompt = SystemMessage(content="You are a helpful AI Assistant. Answer the User's queries succinctly in one sentence.")

# Start Storage for Historical Message History
messages = [system_prompt]

while True:
    # Get User's Message
    user_input = input("\nUser: ")
    
    if user_input.lower() == "exit":
        break

    user_message = HumanMessage(content=user_input)

    # Extend Messages List With User Message
    messages.append(user_message)

    # Pass Entire Message Sequence to LLM to Generate Response
    response = llm.invoke(messages)
    
    print("\nAI Message:", response.content)

    # Add AI's Response to Message List
    messages.append(response)

# Print all messages stored in the message list
for i, message in enumerate(messages):
    print(f"\nMessage {i+1} - {message.type.upper()}: {message.content}")




AI Message: Hello there! How can I help you today?

AI Message: Okay! Nice to meet you, Yash.

AI Message: I’m a large language model, named Gemma.

AI Message: Your name is Yash!

Message 1 - SYSTEM: You are a helpful AI Assistant. Answer the User's queries succinctly in one sentence.

Message 2 - HUMAN: hi

Message 3 - AI: Hello there! How can I help you today?

Message 4 - HUMAN: my name ios yash

Message 5 - AI: Okay! Nice to meet you, Yash.

Message 6 - HUMAN: whtas your name

Message 7 - AI: I’m a large language model, named Gemma.

Message 8 - HUMAN: whats myu name

Message 9 - AI: Your name is Yash!


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

reflection_prompt_template = """
You are analyzing conversations about research papers to create memories that will help guide future interactions. Your task is to extract key elements that would be most helpful when encountering similar academic discussions in the future.

Review the conversation and create a memory reflection following these rules:

1. For any field where you don't have enough information or the field isn't relevant, use "N/A"
2. Be extremely concise - each string should be one clear, actionable sentence
3. Focus only on information that would be useful for handling similar future conversations
4. Context_tags should be specific enough to match similar situations but general enough to be reusable

Output valid JSON in exactly this format:
{{
    "context_tags": [              // 2-4 keywords that would help identify similar future conversations
        string,                    // Use field-specific terms like "deep_learning", "methodology_question", "results_interpretation"
        ...
    ],
    "conversation_summary": string, // One sentence describing what the conversation accomplished
    "what_worked": string,         // Most effective approach or strategy used in this conversation
    "what_to_avoid": string        // Most important pitfall or ineffective approach to avoid
}}

Examples:
- Good context_tags: ["transformer_architecture", "attention_mechanism", "methodology_comparison"]
- Bad context_tags: ["machine_learning", "paper_discussion", "questions"]

- Good conversation_summary: "Explained how the attention mechanism in the BERT paper differs from traditional transformer architectures"
- Bad conversation_summary: "Discussed a machine learning paper"

- Good what_worked: "Using analogies from matrix multiplication to explain attention score calculations"
- Bad what_worked: "Explained the technical concepts well"

- Good what_to_avoid: "Diving into mathematical formulas before establishing user's familiarity with linear algebra fundamentals"
- Bad what_to_avoid: "Used complicated language"

Additional examples for different research scenarios:

Context tags examples:
- ["experimental_design", "control_groups", "methodology_critique"]
- ["statistical_significance", "p_value_interpretation", "sample_size"]
- ["research_limitations", "future_work", "methodology_gaps"]

Conversation summary examples:
- "Clarified why the paper's cross-validation approach was more robust than traditional hold-out methods"
- "Helped identify potential confounding variables in the study's experimental design"

What worked examples:
- "Breaking down complex statistical concepts using visual analogies and real-world examples"
- "Connecting the paper's methodology to similar approaches in related seminal papers"

What to avoid examples:
- "Assuming familiarity with domain-specific jargon without first checking understanding"
- "Over-focusing on mathematical proofs when the user needed intuitive understanding"

Do not include any text outside the JSON object in your response.

Here is the prior conversation:

{conversation}
"""

reflection_prompt = ChatPromptTemplate.from_template(reflection_prompt_template)

reflect = reflection_prompt | llm | JsonOutputParser()


def format_conversation(messages):
    
    # Create an empty list placeholder
    conversation = []
    
    # Start from index 1 to skip the first system message
    for message in messages[1:]:
        conversation.append(f"{message.type.upper()}: {message.content}")
    
    # Join with newlines
    return "\n".join(conversation)

conversation = format_conversation(messages)

print(conversation)



HUMAN: hi
AI: Hello there! How can I help you today?
HUMAN: my name ios yash
AI: Okay! Nice to meet you, Yash.
HUMAN: whtas your name
AI: I’m a large language model, named Gemma.
HUMAN: whats myu name
AI: Your name is Yash!


In [4]:
reflection = reflect.invoke({"conversation": conversation})

print(reflection)

{'context_tags': ['name', 'user_greeting', 'user_introduction'], 'conversation_summary': 'The user introduced themselves as Yash, and the AI confirmed their name is Yash.', 'what_worked': "Leveraging the user's name as a starting point for context establishment and confirming identity.", 'what_to_avoid': 'Assuming a pre-existing knowledge of AI models or language models without explicitly stating it.'}


In [19]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document

# Set up embeddings using Ollama (assuming Ollama is running locally)
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")  # or any embedding model pulled via Ollama

# Example documents
docs = [
    Document(page_content="Understanding attention in transformers."),
    Document(page_content="Control groups are important in experimental setups."),
]

# Create FAISS store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save for later
vectorstore.save_local("faiss_ollama_store")


/tmp/ipykernel_25522/3863004761.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")  # or any embedding model pulled via Ollama


In [25]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document

# 1. Load Ollama Embedding Model
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

# 2. Sample reflections (could be loaded from memory or JSON)
reflections = [
    {
        "title": "Attention Mechanism Explanation",
        "conversation": "USER: How does attention work?\nAI: It's like weighted focus on inputs...",
        "context_tags": ["attention_mechanism", "transformer"],
        "conversation_summary": "Explained attention mechanism in transformers.",
        "what_worked": "Used analogy of human focus to describe attention weights.",
        "what_to_avoid": "Avoided math-heavy explanation without context."
    },
    # Add more items...
]

# 3. Convert to LangChain Document format
docs = []
for item in reflections:
    combined_text = f"""
    Title: {item['title']}
    Conversation: {item['conversation']}
    Summary: {item['conversation_summary']}
    What Worked: {item['what_worked']}
    What to Avoid: {item['what_to_avoid']}
    """
    metadata = {"context_tags": item["context_tags"]}
    docs.append(Document(page_content=combined_text.strip(), metadata=metadata))

# 4. Create FAISS index
faiss_index = FAISS.from_documents(docs, embeddings)

# 5. Save to disk
faiss_index.save_local("faiss_ollama_store")


In [26]:
def add_episodic_memory(messages, vdb_client):

    # Format Messages
    conversation = format_conversation(messages)

    # Create Reflection
    reflection = reflect.invoke({"conversation": conversation})

    # Load Database Collection
    episodic_memory = vdb_client.collections.get("episodic_memory")

    # Insert Entry Into Collection
    episodic_memory.data.insert({
        "conversation": conversation,
        "context_tags": reflection['context_tags'],
        "conversation_summary": reflection['conversation_summary'],
        "what_worked": reflection['what_worked'],
        "what_to_avoid": reflection['what_to_avoid'],
    })


In [28]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

def episodic_recall(query: str, index_path: str = "faiss_ollama_store"):
    # Load Ollama embedding model
    embeddings = OllamaEmbeddings(model="nomic-embed-text")

    # Load FAISS index
    vectorstore = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)

    # Perform similarity search
    results = vectorstore.similarity_search(query, k=1)

    return results


In [29]:
query = "Talking about my name"
memory = episodic_recall(query)

if memory:
    print("Recall result:")
    print(memory[0].page_content)
    print("Context tags:", memory[0].metadata.get("context_tags", []))
else:
    print("No memory found.")


Recall result:
Title: Attention Mechanism Explanation
    Conversation: USER: How does attention work?
AI: It's like weighted focus on inputs...
    Summary: Explained attention mechanism in transformers.
    What Worked: Used analogy of human focus to describe attention weights.
    What to Avoid: Avoided math-heavy explanation without context.
Context tags: ['attention_mechanism', 'transformer']


In [40]:
def episodic_system_prompt(query, vectorstore):
    # Get new memory
    memory = episodic_recall(query, vectorstore)

    current_conversation = memory.objects[0].properties['conversation']
    # Update memory stores, excluding current conversation from history
    if current_conversation not in conversations:
        conversations.append(current_conversation)
    # conversations.append(memory.objects[0].properties['conversation'])
    what_worked.update(memory.objects[0].properties['what_worked'].split('. '))
    what_to_avoid.update(memory.objects[0].properties['what_to_avoid'].split('. '))

    # Get previous conversations excluding the current one
    previous_convos = [conv for conv in conversations[-4:] if conv != current_conversation][-3:]
    
    # Create prompt with accumulated history
    episodic_prompt = f"""You are a helpful AI Assistant. Answer the user's questions to the best of your ability.
    You recall similar conversations with the user, here are the details:
    
    Current Conversation Match: {memory.objects[0].properties['conversation']}
    Previous Conversations: {' | '.join(previous_convos)}
    What has worked well: {' '.join(what_worked)}
    What to avoid: {' '.join(what_to_avoid)}
    
    Use these memories as context for your response to the user."""
    
    return SystemMessage(content=episodic_prompt)

In [41]:
conversations = []
what_worked = set()
what_to_avoid = set()

# Start Storage for Historical Message History
messages = []

while True:
    # Get User's Message
    user_input = input("\nUser: ")
    user_message = HumanMessage(content=user_input)
    
    # Generate new system prompt
    system_prompt = episodic_system_prompt(user_input, vectorstore)
    
    # Reconstruct messages list with new system prompt first
    messages = [
        system_prompt,  # New system prompt always first
        *[msg for msg in messages if not isinstance(msg, SystemMessage)]  # Old messages except system
    ]
    
    if user_input.lower() == "exit":
        add_episodic_memory(messages, vectorstore)
        print("\n == Conversation Stored in Episodic Memory ==")
        break
    if user_input.lower() == "exit_quiet":
        print("\n == Conversation Exited ==")
        break
    
    # Add current user message
    messages.append(user_message)
    
    # Pass Entire Message Sequence to LLM to Generate Response
    response = llm.invoke(messages)
    print("\nAI Message: ", response.content)
    
    # Add AI's Response to Message List
    messages.append(response)


AttributeError: 'OllamaEmbeddings' object has no attribute 'encode'